In [2]:
import pandas as pd
import numpy as np
import spacy
import re
import random
from tqdm import tqdm
from pathlib import Path
import nltk
from nltk.stem import PorterStemmer
import glob
import os

# File Stitcher - RUN THIS CODE ONCE!

There are 350k files in the data folder, which takes forever for the computer to go through. Also, there is "all parents" csv but some of the individual parent files are not included. So for convenience and having a complete set we will first stitch all the raw data into a single file: 

In [6]:
# DATA DIR
data_dir = Path("data")

def build_master_csv(file_pattern, output_name, id_column='id'):

    print(f"Looking for {file_pattern} files.")
    # Get list of all files matching the pattern
    # For children, we exclude the '_output.csv' files which are legacy matches
    all_files = glob.glob(str(data_dir / file_pattern))
    
    # Filter out 'output' files if we are looking for children
    if 'c_' in file_pattern:
        all_files = [f for f in all_files if '_output.csv' not in f]
    
    print(f"Found {len(all_files)} files. Merging them into {output_name}.")
    
    # Use a list to store dataframes (faster than appending to a dataframe)
    df_list = []
    
    for filename in tqdm(all_files):
        try:
            # Read only the columns we need to save memory
            # Adjust columns based on what your new_preprocess.ipynb needs
            df = pd.read_csv(filename, index_col=None, header=0)
            df_list.append(df)
        except Exception as e:
            print(f"Error reading {filename}: {e}")
            continue
            
    if df_list:
        master_df = pd.concat(df_list, axis=0, ignore_index=True)
        
        # Deduplicate just in case
        if id_column in master_df.columns:
            master_df = master_df.drop_duplicates(subset=[id_column])
            master_df = master_df.set_index(id_column) # Set ID as index for faster lookups
            
        # Save to CSV
        output_path = data_dir / output_name
        master_df.to_csv(output_path)
        print(f"Success! Saved {output_name} with {len(master_df)} rows.")
        return master_df
    else:
        print(f"No files found for {file_pattern}!")
        return None

# Build full_children.csv (formerly childs_from_id.csv)
build_master_csv("c_*.csv", "full_children.csv")

# Build full_parents.csv (merges individual p_ files AND the original all_parents.csv input)
existing_parents = []
if (data_dir / "all_parents.csv").exists():
    print("Loading existing all_parents.csv to merge.")
    existing_parents.append(pd.read_csv(data_dir / "all_parents.csv"))

# Get all p_*.csv files
p_files = glob.glob(str(data_dir / "p_*.csv"))
print(f"Found {len(p_files)} individual parent files.")

for f in tqdm(p_files):
    existing_parents.append(pd.read_csv(f))

if existing_parents:
    full_parents = pd.concat(existing_parents, axis=0, ignore_index=True)
    # Deduplicate by ID
    if 'id' in full_parents.columns:
        full_parents = full_parents.drop_duplicates(subset=['id'])
        full_parents = full_parents.set_index('id')
    
    # Save to NEW name
    full_parents.to_csv(data_dir / "full_parents.csv")
    print(f"Success! Saved full_parents.csv. Total parents: {len(full_parents)}")

Looking for c_*.csv files.
Found 174190 files. Merging them into full_children.csv.


100%|██████████| 174190/174190 [04:30<00:00, 644.96it/s]


Success! Saved full_children.csv with 174190 rows.
Loading existing all_parents.csv to merge.
Found 1594 individual parent files.


100%|██████████| 1594/1594 [00:02<00:00, 596.06it/s]


Success! Saved full_parents.csv. Total parents: 2656


Carry on with rest of the processing:
# SAFE TO RUN FROM THIS CELL AND DOWN

In [10]:
# Load the large Dutch model (For Model B)
# If this crashes, Step 1 didn't work.
nlp = spacy.load("nl_core_news_lg", disable=['ner', 'parser'])
print("Success: Model loaded!")

# Define paths
data_path = Path("data")

# The English Stemmer for Model A (Make sure you imported nltk!)
ps = PorterStemmer()

Success: Model loaded!


In [11]:
#!python -m spacy download nl_core_news_lg

Added Dutch SpaCy Lemmatization to PorterStemmer to actually capture Dutch meaning.

In [12]:
# Dutch cleaning (Model B)
def clean_text_dutch(text):
    if not isinstance(text, str): return ""
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text) 
    doc = nlp(text)
    # Keep lemmas (Dutch dictionary roots)
    tokens = [token.lemma_ for token in doc if not token.is_stop]
    return " ".join(tokens)

# Legacy English cleaning (Model A)
def clean_text_legacy(text):
    if not isinstance(text, str): return ""
    text = text.lower()

    # Simple regex
    text = re.sub(r'[^a-z0-9\s]', '', text)
    tokens = text.split()
    stems = [ps.stem(t) for t in tokens]
    return " ".join(stems)

# CALculating the similarity score:
def jaccard_similarity(str1, str2):
    """Legacy metric: Intersection over Union"""
    a = set(str1.split()) 
    b = set(str2.split())
    if len(a) == 0 or len(b) == 0: return 0.0
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

Cleaning the dataframes before the matching loop so we don't clean the same text 10,000 times.

In [ ]:
print("Loading raw CSVs, good luck to our laptops.")
children_df = pd.read_csv(data_path / 'full_children.csv', index_col=0).fillna('')
parents_df = pd.read_csv(data_path / 'full_parents.csv', index_col=0).fillna('')

# Force indices to be integers to ensure matching works
children_df.index = pd.to_numeric(children_df.index, errors='coerce').fillna(-1).astype(int)
parents_df.index = pd.to_numeric(parents_df.index, errors='coerce').fillna(-1).astype(int)

# Model B / Dutch
children_df['clean_title_dutch'] = children_df['title'].astype(str).apply(clean_text_dutch)
children_df['clean_content_dutch'] = children_df['content'].astype(str).apply(clean_text_dutch)
parents_df['clean_title_dutch'] = parents_df['title'].astype(str).apply(clean_text_dutch)
parents_df['clean_content_dutch'] = parents_df['content'].astype(str).apply(clean_text_dutch)

# Model A / English
children_df['clean_title_legacy'] = children_df['title'].astype(str).apply(clean_text_legacy)
children_df['clean_content_legacy'] = children_df['content'].astype(str).apply(clean_text_legacy)
parents_df['clean_title_legacy'] = parents_df['title'].astype(str).apply(clean_text_legacy)
parents_df['clean_content_legacy'] = parents_df['content'].astype(str).apply(clean_text_legacy)

print("Data loaded and cleaned.")

Loading raw CSVs, good luck to our laptops.


/var/folders/r5/_05_w2k14p728pkkf57ysjg40000gn/T/ipykernel_83559/1321362252.py:2: DtypeWarning: Columns (15,16,56,57,61,62,68,69,72) have mixed types. Specify dtype option on import or set low_memory=False.
  children_df = pd.read_csv(data_path / 'full_children.csv', index_col=0).fillna('')


Data loaded and cleaned.


The old script had issues with label overwriting. I wrote this loop to explicitly extract matches from the 'related_children' column and generate exactly 1 random negative for every positive.

In [ ]:
training_rows = []

# List of all available parent IDs for random sampling
all_parent_ids = parents_df.index.tolist()

print("Building training pairs...")

for parent_id, row in tqdm(parents_df.iterrows(), total=len(parents_df)):
    
    # --- THE FIX IS HERE ---
    # Use Regex to extract ALL numbers (IDs) from the messy string.
    # This works for "123, 456", "['123', '456']", and "matches/123" all at once.
    child_ids = re.findall(r'\d+', str(row['related_children']))
    
    # If no IDs found, skip
    if not child_ids:
        continue
        
    for child_id in child_ids:
        try:
            c_id = int(child_id) # Convert extracted string to integer
            
            # Check if this child actually exists in our child database
            if c_id in children_df.index:
                
                # --- CREATE POSITIVE SAMPLE (Match = 1) ---
                training_rows.append({
                    'child_id': c_id,
                    'parent_id': parent_id,
                    'match': 1
                })
                
                # --- CREATE NEGATIVE SAMPLE (Match = 0) ---
                # Pick a random parent that is NOT the current parent
                while True:
                    random_pid = random.choice(all_parent_ids)
                    if random_pid != parent_id:
                        break
                
                training_rows.append({
                    'child_id': c_id,
                    'parent_id': random_pid,
                    'match': 0
                })
                
        except ValueError:
            continue

# Convert to DataFrame
trainset_new = pd.DataFrame(training_rows)
print(f"Generated {len(trainset_new)} rows (Balanced 50/50).")

Building training pairs.


100%|██████████| 2656/2656 [00:00<00:00, 28336.07it/s]

Generated 10 rows (Balanced 50/50).


Using nlp.pipe to vectorize everything in batches. This is much faster than .iterrows().

In [15]:
# MOdel B
# Map IDs to Dutch text
c_titles_d = [children_df.at[cid, 'clean_title_dutch'] if cid in children_df.index else "" for cid in trainset_new['child_id']]
p_titles_d = [parents_df.at[pid, 'clean_title_dutch'] if pid in parents_df.index else "" for pid in trainset_new['parent_id']]
c_content_d = [children_df.at[cid, 'clean_content_dutch'] if cid in children_df.index else "" for cid in trainset_new['child_id']]
p_content_d = [parents_df.at[pid, 'clean_content_dutch'] if pid in parents_df.index else "" for pid in trainset_new['parent_id']]

print("Calculating Vectors.")

# Batch process with SpaCy
c_title_docs = list(nlp.pipe(c_titles_d, batch_size=200))
p_title_docs = list(nlp.pipe(p_titles_d, batch_size=200))
c_cont_docs = list(nlp.pipe(c_content_d, batch_size=50))
p_cont_docs = list(nlp.pipe(p_content_d, batch_size=50))

trainset_new['title_sim_dutch'] = [c.similarity(p) if c.vector_norm and p.vector_norm else 0.0 for c, p in zip(c_title_docs, p_title_docs)]
trainset_new['content_sim_dutch'] = [c.similarity(p) if c.vector_norm and p.vector_norm else 0.0 for c, p in zip(c_cont_docs, p_cont_docs)]

# For model A
print("Calculating Legacy Jaccard.")

# Map IDs to Legacy text
c_titles_l = [children_df.at[cid, 'clean_title_legacy'] if cid in children_df.index else "" for cid in trainset_new['child_id']]
p_titles_l = [parents_df.at[pid, 'clean_title_legacy'] if pid in parents_df.index else "" for pid in trainset_new['parent_id']]
c_content_l = [children_df.at[cid, 'clean_content_legacy'] if cid in children_df.index else "" for cid in trainset_new['child_id']]
p_content_l = [parents_df.at[pid, 'clean_content_legacy'] if pid in parents_df.index else "" for pid in trainset_new['parent_id']]

# Simple list comprehension for Jaccard
trainset_new['title_sim_legacy'] = [jaccard_similarity(c, p) for c, p in zip(c_titles_l, p_titles_l)]
trainset_new['content_sim_legacy'] = [jaccard_similarity(c, p) for c, p in zip(c_content_l, p_content_l)]

print("Scoring complete.")

Calculating Vectors.
Calculating Legacy Jaccard.
Scoring complete.


Re-implementing the date binary check. I am omitting the complex taxonomy features for now as they require the legacy dictionary logic, but we have the core semantic features.

In [16]:
# Add Date Logic
# (We need to look up dates from the original dfs)
c_dates = children_df.loc[trainset_new['child_id']]['publish_date']
p_dates = parents_df.loc[trainset_new['parent_id']]['publish_date']

# Convert to datetime
c_dates = pd.to_datetime(c_dates, errors='coerce').dt.tz_localize(None)
p_dates = pd.to_datetime(p_dates, errors='coerce').dt.tz_localize(None)

# Calculate difference in days
trainset_new['days_diff'] = (c_dates.values - p_dates.values) / np.timedelta64(1, 'D')
trainset_new['days_diff'] = trainset_new['days_diff'].abs()

# Create the binary feature (legacy logic was a window of 2 days)
trainset_new['date_binary'] = (trainset_new['days_diff'] <= 2).astype(int)

print(trainset_new.head())

# Save the reconstructed set
output_filename = 'trainset_reconstructed.csv'
trainset_new.to_csv(output_filename, index=False)
print(f"Saved reconstructed training set to {output_filename}")

   child_id  parent_id  match  title_sim_dutch  content_sim_dutch  \
0    955090     954939      1         1.000000           0.985670   
1    955090    1458994      0         0.575754           0.718978   
2    955140     954939      1         0.874414           0.662531   
3    955140    1379022      0         0.410175           0.843156   
4    955703     954939      1         0.668406           0.876471   

   title_sim_legacy  content_sim_legacy  days_diff  date_binary  
0          1.000000            0.735294   0.036921            1  
1          0.000000            0.051546        NaN            0  
2          0.555556            0.112108   0.590278            1  
3          0.071429            0.135802        NaN            0  
4          0.062500            0.151899   0.773819            1  
Saved reconstructed training set to trainset_reconstructed.csv
